In [1]:
import pickle as pk

dumppath = '/home/lscsc/caizhijie/0420-wamera-benchmark/validdumps_80epoch.pk'
arr = pk.load(open(dumppath, 'rb'))

df = pk.load(open('/home/lscsc/caizhijie/0420-wamera-benchmark/annotate/sorted_annotate_valid.pk', 'rb'))

/home/lscsc/caizhijie/miniconda3/envs/mtmd/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
arr[0]

'i, loss_sm, loss_jhm, loss_paf, sm, jhm, paf, y_sm, y_jhm, y_paf, img, box'

In [3]:
loss_sm_list = list()
loss_jhm_list = list()
loss_paf_list = list()
sm_list = list()
jhm_list = list()
paf_list = list()
y_sm_list = list()
y_jhm_list = list()
y_paf_list = list()
img_list = list()
box_list = list()

for i in arr[1:40]:
    loss_sm_list.append(i[1])
    loss_jhm_list.append(i[2])
    loss_paf_list.append(i[3])
    sm_list.extend(i[4])
    jhm_list.extend(i[5])
    paf_list.extend(i[6])
    y_sm_list.extend(i[7])
    y_jhm_list.extend(i[8])
    y_paf_list.extend(i[9])
    img_list.extend(i[10])
    box_list.extend(i[11])

In [4]:
import numpy as np
import sys

sys.path.append('../src')

import utils

n_color = 19
colors, hls = utils.ncolors(n_color)
colors[18] = [0, 0, 0]

jhm_arr = np.stack(jhm_list)
y_jhm_arr = np.stack(y_jhm_list)
paf_arr = np.stack(paf_list)
y_paf_arr = np.stack(y_paf_list)
sm_arr = np.stack(sm_list)
y_sm_arr = np.stack(y_sm_list)

In [5]:
y_jhm_arr.shape

(1248, 19, 46, 46)

In [6]:
img_list[0]

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [7]:
import cv2
import tqdm
import previewer

left = 200
right = 600

def previewvideo(df, left, right, jhm, paf, y_jhm, y_paf, pd=None, dumppath=''):
    read_imgs = list(df.iloc[left:right]['jpg'].apply(lambda x: cv2.imread(x)).values)
    
    jhm = jhm
    y_jhm = y_jhm
    paf = paf
    y_paf = y_paf
    sm = sm_arr[left:right, ...]
    y_sm = y_sm_arr[left:right, ...]
    
    if pd is None:
        img_list = list()
        for i in tqdm.trange(0, right-left, 32):
            imgbatch = np.zeros((sm.shape[0], 512, 512))
            pd = previewer.previewbatch(imgbatch, (y_jhm[i:i+32], y_paf[i:i+32]), (jhm[i:i+32], paf[i:i+32]))
            img_list.extend(pd)
    else:
        fake = pd[::2, ...]
        real = pd[1::2, ...]
        img_list = np.concatenate([fake, real], 2)
    
    jhm = jhm[left:right, ...] / np.expand_dims(jhm[left:right, ...].sum(1), 1)
    y_jhm = y_jhm[left:right, ...] / np.expand_dims(y_jhm[left:right, ...].sum(1), 1)
    
    sm = np.repeat(np.expand_dims(sm, 1), 3, 1).transpose(0, 2, 3, 1)
    y_sm = np.repeat(np.expand_dims(y_sm, 1), 3, 1).transpose(0, 2, 3, 1)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowrite = cv2.VideoWriter(dumppath + 'pd.mp4', fourcc, 20, (1024, 512))
    for i in tqdm.trange(read_imgs.__len__()):
        videowrite.write(np.uint8(img_list[i, ..., ::1]))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowrite = cv2.VideoWriter(dumppath + 'gt.mp4', fourcc, 20, (512, 512))
    for i in tqdm.trange(read_imgs.__len__()):
        videowrite.write(read_imgs[i][..., ::1])
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowrite = cv2.VideoWriter(dumppath + 'jhm.mp4', fourcc, 20, (46, 46))
    jhm = np.matmul(jhm.transpose(0, 2, 3, 1), np.array(colors) / 255) * 255
    
    for i in tqdm.trange(read_imgs.__len__()):
        videowrite.write(np.uint8(jhm[i, ...]))
        
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowrite = cv2.VideoWriter(dumppath + 'y_jhm.mp4', fourcc, 20, (46, 46))
    y_jhm = np.matmul(y_jhm.transpose(0, 2, 3, 1), np.array(colors) / 255) * 255
    for i in tqdm.trange(read_imgs.__len__()):
        videowrite.write(np.uint8(y_jhm[i, ...]))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowrite = cv2.VideoWriter(dumppath + 'sm.mp4', fourcc, 20, (46, 46))
    for i in tqdm.trange(read_imgs.__len__()):
        videowrite.write(np.uint8(sm[i, ...] * 255))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videowrite = cv2.VideoWriter(dumppath + 'y_sm.mp4', fourcc, 20, (46, 46))
    for i in tqdm.trange(read_imgs.__len__()):
        videowrite.write(np.uint8(y_sm[i, ...] * 255))


In [8]:
previewvideo(df, 0, 800, jhm_arr, paf_arr, y_jhm_arr, y_paf_arr, np.stack(img_list), dumppath=dumppath.split('/')[-1][:-3] + '_')

100%|██████████| 800/800 [00:00<00:00, 5602.00it/s]
